In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
 
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)
 
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#卷积去噪自编码器

### 编码器
conv1 = tf.layers.conv2d(inputs_, 32, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 28x28x32
maxpool1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), padding='same')
# 当前shape: 14x14x32
conv2 = tf.layers.conv2d(maxpool1, 32, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 14x14x32
maxpool2 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
# 当前shape: 7x7x32
conv3 = tf.layers.conv2d(maxpool2, 16, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 7x7x16
encoded = tf.layers.max_pooling2d(conv3, (2,2), (2,2), padding='same')
# 当前shape: 4x4x16
 
 
### 解码器
upsample1 = tf.image.resize_nearest_neighbor(encoded, (7,7))
# 当前shape: 7x7x16
conv4 = tf.layers.conv2d(upsample1, 16, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 7x7x16
upsample2 = tf.image.resize_nearest_neighbor(conv4, (14,14))
# 当前shape: 14x14x16
conv5 = tf.layers.conv2d(upsample2, 32, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 14x14x32
upsample3 = tf.image.resize_nearest_neighbor(conv5, (28,28))
# 当前shape: 28x28x32
conv6 = tf.layers.conv2d(upsample3, 32, (3,3), padding='same', activation=tf.nn.relu)
# 当前shape: 28x28x32
 
 
logits = tf.layers.conv2d(conv6, 1, (3,3), padding='same', activation=None)
#当前shape: 28x28x1
 
 
decoded = tf.nn.sigmoid(logits, name='decoded')
 
 
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=targets_, logits=logits)
cost = tf.reduce_mean(loss)
opt = tf.train.AdamOptimizer(0.001).minimize(cost)
 
 
sess = tf.Session()
epochs = 100
batch_size = 20

sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range(mnist.train.num_examples // batch_size):
        batch = mnist.train.next_batch(batch_size)
        # 
        imgs = batch[0].reshape((-1, 28, 28, 1))
        # 
        noisy_imgs = imgs + noise_factor * np.random.randn(*imgs.shape)
        # 
        noisy_imgs = np.clip(noisy_imgs, 0., 1.)
        # 
        batch_cost, _ = sess.run([cost, opt], feed_dict={inputs_: noisy_imgs,
                                                         targets_: imgs})
        print("Epoch: {}/{}...".format(e + 1, epochs),
              "Training loss: {:.4f}".format(batch_cost))
 
 
in_imgs = mnist.test.images[:10]
noisy_imgs = in_imgs + noise_factor * np.random.randn(*in_imgs.shape)
noisy_imgs = np.clip(noisy_imgs, 0., 1.)
reconstructed = sess.run(decoded, feed_dict={inputs_: noisy_imgs.reshape((10, 28, 28, 1))})
plt.figure(figsize=(20,4))
for i in range(10):
    #画原始图像
    ax=plt.subplot(2,10,i+1)
    plt.imshow(in_imgs[i].reshape((28,28)))
    plt.gray()#只有黑白两色，没有中间色
    #重建图像
    ax=plt.subplot(2,10,i+10+1)
    plt.imshow(reconstructed[i].reshape((28,28)))
    plt.gray()
    ax.get_xaxis().set_visible(False) #不显示x轴
    ax.get_yaxis().set_visible(False)#不显示y轴
plt.show()
sess.close()

NameError: name 'noise_factor' is not defined

In [4]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf

import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
 
mnist = input_data.read_data_sets('MNIST_data', validation_size=0)
 
inputs_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='inputs')
targets_ = tf.placeholder(tf.float32, (None, 28, 28, 1), name='targets')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
#实现标准的均匀分布的Xaiver初始化器，其中fan_in是输入节点的数量，fan_out是输出节点的数量
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
        minval = low, maxval = high, dtype = tf.float32)

In [9]:
#定义一个去噪自编码的类，方便使用。

class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus, optimizer = tf.train.AdamOptimizer(), scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights

In [12]:
#卷积神经网络
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
mnist = input_data.read_data_sets( 'MNIST_data', one_hot=True, reshape=False)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [16]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    conv1_weight = tf.get_variable('c1_w', [5, 5, 1, 8], dtype=tf.float32) #权重
    conv1_bias = tf.get_variable('c1_b', [8], dtype=tf.float32)  #偏置量
    conv1_ = tf.nn.conv2d(inputs, conv1_weight, [1, 1, 1, 1], 'VALID')
    
    conv1 = tf.nn.relu(conv1_ + conv1_bias)
    
    # shape=[None, 12, 12, 8]
    pool1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    conv2_weight = tf.get_variable('c2_w', shape=[5, 5, 8, 16], dtype=tf.float32)
    conv2_bias = tf.get_variable( 'c2_b', shape=[16], dtype=tf.float32)
    conv2_ = tf.nn.conv2d(pool1, conv2_weight, [1, 1, 1, 1], 'VALID')
    # shape=[None, 8, 8, 16]
    conv2 = tf.nn.relu(conv2_ + conv2_bias)
    
    # shape=[None, 4, 4, 16]
    pool2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    
    tmp = tf.reshape(pool2, [-1, 4 * 4 * 16])
    
    logits = tf.keras.layers.Dense(10, activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    saver = tf.train.Saver()
    
    # 代价函数
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
    
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.4244, acc 0.0518
step   500, loss 0.5025, acc 0.8786
step  1000, loss 0.1703, acc 0.9270
step  1500, loss 0.1255, acc 0.9440
step  2000, loss 0.1135, acc 0.9533
step  2500, loss 0.1612, acc 0.9539
step  3000, loss 0.0938, acc 0.9619
step  3500, loss 0.2240, acc 0.9645
step  4000, loss 0.0918, acc 0.9648
step  4500, loss 0.1278, acc 0.9670
step  5000, loss 0.0393, acc 0.9714
step  5500, loss 0.0999, acc 0.9724
step  6000, loss 0.0567, acc 0.9707
step  6500, loss 0.0091, acc 0.9732
step  7000, loss 0.0499, acc 0.9733
step  7500, loss 0.0205, acc 0.9735
step  8000, loss 0.0637, acc 0.9733
step  8500, loss 0.0684, acc 0.9741
step  9000, loss 0.0899, acc 0.9742
step  9500, loss 0.1292, acc 0.9763
step 10000, loss 0.0112, acc 0.9788
step 10500, loss 0.0375, acc 0.9770
step 11000, loss 0.1680, acc 0.9765
step 11500, loss 0.0490, acc 0.9761
step 12000, loss 0.1508, acc 0.9784
step 12500, loss 0.0694, acc 0.9765
step 13000, loss 0.1314, acc 0.9801
step 13500, loss 0.0401, acc